# Using Custom Artifact Visualizations with InteractiveContext

This notebook is a companion to a [blog post by Suzen Fylke](https://codesue.com/blog/how-to-visualize-custom-tfx-artifacts-with-interactivecontext/).

*The snippets from the TFX Keras Component Tutorial and the InteractiveContext.show() definition used in this notebook are Copyright 2021 by The TensorFlow Authors and licensed under the Apache License, Version 2.0.*

## How InteractiveContext Visualizes Artifacts

When you call `InteractiveContext.show()` on an artifact, `InteractiveContext` checks whether there's a visualization for the artifact in its visualization registry. Then, it calls the visualization's `display` method if a visualization is available.

Here is the [source code](https://github.com/tensorflow/tfx/blob/b65ef49815aeb8baa58f633930c0c55081d2ede9/tfx/orchestration/experimental/interactive/interactive_context.py#L222) for `InteractiveContext.show()`:

```python
from tfx.orchestration.experimental.interactive import notebook_utils
from tfx.orchestration.experimental.interactive import visualizations


@notebook_utils.requires_ipython
def show(self, item: object) -> None:
  """Show the given object in an IPython notebook display."""
  from IPython.core.display import display
  from IPython.core.display import HTML
  if isinstance(item, types.Channel):
    channel = item
    artifacts = channel.get()
    for artifact in artifacts:
      artifact_heading = 'Artifact at %s' % html.escape(artifact.uri)
      display(HTML('<b>%s</b><br/><br/>' % artifact_heading))
      visualization = visualizations.get_registry().get_visualization(
          artifact.type_name)
      if visualization:
        visualization.display(artifact)
  else:
    display(item)
```

Default visualizations are defined in the [standard_visualizations module](https://github.com/tensorflow/tfx/blob/b65ef49815aeb8baa58f633930c0c55081d2ede9/tfx/orchestration/experimental/interactive/standard_visualizations.py), and they're registered in the [InteractiveContext's constructor](https://github.com/tensorflow/tfx/blob/b65ef49815aeb8baa58f633930c0c55081d2ede9/tfx/orchestration/experimental/interactive/interactive_context.py#L109).

## How to Create Custom Visualizations

To create a custom visualization:

- Subclass the `ArtifactVisualization` abstract base class.
- Override the `ARTIFACT_TYPE` property with the type of artifact the visualization applies to.
- Override the `display` method to read relevant content from the artifact's URI and to render the content.

When you're ready to display your artifact:

- Add the visualization to `InteractiveContext`'s visualization registry.
- Run `InteractiveContext.show()` on the artifact.

## Let's Walk Through an Example!

We'll run the `ExampleGen` component from the [TFX Keras Component tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) and take a look at the first three training examples it produces. We'll start by pretty printing the examples like the tutorial does, then we'll refactor that code to create a custom visualization.

### Install TFX and restart the runtime.

In [ ]:
!pip install tfx

### Set up and run the ExampleGen component.

In [1]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()
absl.logging.set_verbosity(absl.logging.INFO)

DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

context = InteractiveContext()
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-dataapptptbz/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Pretty print the training examples

Pretty print the training examples as is done in the TFX Keras Component tutorial.

In [2]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Chicago Elite Cab Corp. (Chicago Carriag"
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 12.449999809265137
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      bytes_list {
        value: "Credit Card"
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
   

### Create an ExampleVisualization

Here we create a custom visualization to display `Examples` artifacts with `InteractiveContext`.

First, we subclass `ArtifactVisualization`. Then, we set `ARTIFACT_TYPE` to `Examples`. Finally, we create a `display` method that reads training examples from an artifact's URI and pretty prints them. We reuse the TFX Keras Component tutorial's code for pretty printing training examples in the display method.

In [3]:
from tfx.orchestration.experimental.interactive import visualizations
from tfx.types import artifact, standard_artifacts


class ExampleVisualization(visualizations.ArtifactVisualization):
  ARTIFACT_TYPE = standard_artifacts.Examples

  def display(self, artifact: artifact.Artifact):
    
    # Get the URI of the output artifact representing the training examples, which is a directory
    train_uri = os.path.join(artifact.uri, 'Split-train')

    # Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(train_uri, name)
                          for name in os.listdir(train_uri)]

    # Create a `TFRecordDataset` to read these files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

    # Iterate over the first 3 records and decode them.
    for tfrecord in dataset.take(3):
      serialized_example = tfrecord.numpy()
      example = tf.train.Example()
      example.ParseFromString(serialized_example)
      pp.pprint(example)


Add the `ExampleVisualization` to `InteractiveContext`'s visualization registry.

In [4]:
visualizations.get_registry().register(ExampleVisualization)

Display the examples.


In [5]:
context.show(example_gen.outputs['examples'])

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: "Chicago Elite Cab Corp. (Chicago Carriag"
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 12.449999809265137
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      bytes_list {
        value: "Credit Card"
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
   

We can use the same visualization we just created for transformed examples as well:

```python
from tfx.components import Transform


transform = Transform(...)
context.run(transform, enable_cache=True)
context.show(transform.outputs['transformed_examples'])
```